In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.size'] = 18
%matplotlib inline

N_FOLDS = 5
MAX_EVALS = 5

In [2]:
features = pd.read_csv('D:\statistics\machine learning in action\Kaggle-Home-Credit-Default-Risk-Solution-master\Model\m_train_small.csv')
labels = np.array(features['TARGET'].astype(np.int32)).reshape((-1, ))
features = features.drop(columns = ['TARGET', 'SK_ID_CURR'])



KeyboardInterrupt: 

In [3]:
features['CREDIT_INCOME_PERCENT'] = features['AMT_CREDIT'] / features['AMT_INCOME_TOTAL']
features['ANNUITY_INCOME_PERCENT'] = features['AMT_ANNUITY'] / features['AMT_INCOME_TOTAL']
features['CREDIT_TERM'] = features['AMT_ANNUITY'] / features['AMT_CREDIT']
features['DAYS_EMPLOYED_PERCENT'] = features['DAYS_EMPLOYED'] / features['DAYS_BIRTH']

In [4]:
# Split into training and testing data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 6000, random_state = 42)

print('Train shape: ', train_features.shape)
print('Test shape: ', test_features.shape)

train_features.head()

Train shape:  (301511, 338)
Test shape:  (6000, 338)


,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,client_installments_AMT_PAYMENT_min_sum,AMT_CREDIT,AMT_ANNUITY,DAYS_EMPLOYED,bureau_DAYS_CREDIT_max,client_cash_CNT_INSTALMENT_FUTURE_min_max,...,previous_NAME_SELLER_INDUSTRY_Construction_mean,previous_NAME_TYPE_SUITE_Other_A_mean,client_credit_AMT_RECIVABLE_min_sum,client_bureau_balance_STATUS_X_count_sum,client_credit_AMT_DRAWINGS_CURRENT_max_sum,client_cash_CNT_INSTALMENT_min_max,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
193071,0.603835,0.720645,0.819318,-20060,1393669.80,215865.0,22797.0,365243,-875.0,16.0,...,0.333333,0.0,NaN,2.0,NaN,36.0,0.872182,0.092109,0.105608,-18.207527
302257,NaN,0.416338,0.607557,-11873,106383.78,675000.0,34596.0,-3862,-168.0,0.0,...,0.000000,0.0,NaN,43.0,NaN,18.0,3.750000,0.192200,0.051253,0.325276
263273,NaN,0.730373,0.379100,-24987,441598.50,485640.0,31531.5,365243,-213.0,7.0,...,0.000000,0.0,NaN,NaN,NaN,12.0,2.398222,0.155711,0.064928,-14.617321
68522,NaN,0.520688,0.387625,-23965,49676.85,552555.0,16879.5,365243,-195.0,0.0,...,0.111111,0.0,NaN,NaN,NaN,6.0,3.069750,0.093775,0.030548,-15.240684
6848,0.497775,0.373383,NaN,-16137,189557.55,665325.0,35577.0,-497,-2397.0,4.0,...,0.000000,0.0,-541928.94,NaN,1539000.0,12.0,4.769355,0.255032,0.053473,0.030799


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [10]:
from sklearn.impute import SimpleImputer

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(train_features)
train_features=imputer.transform(train_features)
test_features=imputer.transform(test_features)

In [16]:
rfc = RandomForestClassifier(n_estimators=100,random_state=90)
score_pre = cross_val_score(rfc,train_features,train_labels,cv=5).mean()
score_pre

0.919307502206762

In [17]:
rfc = RandomForestClassifier(n_estimators=50,random_state=90)
score_pre = cross_val_score(rfc,train_features,train_labels,cv=5).mean()
score_pre

0.9193196980722395

In [ ]:
param_grid = {'min_samples_split':np.arange(2, 2+20, 5)}

param_grid = {'min_samples_leaf':np.arange(1, 1+10, 5)}
    
param_grid = {'max_features':np.arange(5,30,10)}

In [19]:
param_grid = {'max_depth':np.arange(1, 8, 3)}
rfc = RandomForestClassifier(n_estimators=50
                             ,random_state=90
                            )
GS = GridSearchCV(rfc,param_grid,cv=3)
GS.fit(train_features,train_labels)

GS.best_params_

GS.best_score_

0.919205879483594

In [20]:
print(GS.best_params_)

{'max_depth': 1}


In [21]:
param_grid = {'criterion':['gini', 'entropy']}

rfc = RandomForestClassifier(n_estimators=39
                             ,random_state=90,max_depth=2
                            )
GS = GridSearchCV(rfc,param_grid,cv=3)
GS.fit(train_features,train_labels)

print(GS.best_params_)

GS.best_score_

{'criterion': 'gini'}


0.919205879483594

In [3]:
features = pd.read_csv('D:\statistics\machine learning in action\Kaggle-Home-Credit-Default-Risk-Solution-master\Model\m_train_small.csv')
test_featurees=pd.read_csv('D:\statistics\machine learning in action\Kaggle-Home-Credit-Default-Risk-Solution-master\Model\m_train_small.csv')



In [5]:
features['CREDIT_INCOME_PERCENT'] = features['AMT_CREDIT'] / features['AMT_INCOME_TOTAL']
features['ANNUITY_INCOME_PERCENT'] = features['AMT_ANNUITY'] / features['AMT_INCOME_TOTAL']
features['CREDIT_TERM'] = features['AMT_ANNUITY'] / features['AMT_CREDIT']
features['DAYS_EMPLOYED_PERCENT'] = features['DAYS_EMPLOYED'] / features['DAYS_BIRTH']

In [7]:
test_featurees['CREDIT_INCOME_PERCENT'] = test_featurees['AMT_CREDIT'] / test_featurees['AMT_INCOME_TOTAL']
test_featurees['ANNUITY_INCOME_PERCENT'] = test_featurees['AMT_ANNUITY'] / test_featurees['AMT_INCOME_TOTAL']
test_featurees['CREDIT_TERM'] = test_featurees['AMT_ANNUITY'] / test_featurees['AMT_CREDIT']
test_featurees['DAYS_EMPLOYED_PERCENT'] = test_featurees['DAYS_EMPLOYED'] / test_featurees['DAYS_BIRTH']

In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(features)
features=imputer.transform(features)
test_features=imputer.transform(test_featurees)

MemoryError: 

In [4]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    labels = features['TARGET']
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)        
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        cat_indices = 'auto'
        
    elif encoding == 'le':        
        label_encoder = LabelEncoder()        
        cat_indices = []
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))
                cat_indices.append(i)
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    feature_names = list(features.columns)
    features = np.array(features)
    test_features = np.array(test_features)
    
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    feature_importance_values = np.zeros(len(feature_names))
    
    test_predictions = np.zeros(test_features.shape[0])
    
    train_predictions=np.zeros(features.shape[0])
    
    out_of_fold = np.zeros(features.shape[0])
    
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        train_features, train_labels = features[train_indices], labels[train_indices]
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        model = RandomForestClassifier(n_estimators=50, 
                                   max_depth=2,
                                   creterion='gini', n_jobs = -1, random_state = 90)
        
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,)
        

        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        test_predictions += model.predict_proba(test_features)[:, 1] / k_fold.n_splits
        out_of_fold[valid_indices] = model.predict_proba(valid_features)[:, 1]
        
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        

    submission1 = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    submission2 = pd.DataFrame({'SK_ID_CURR': train_ids, 'TARGET': out_of_fold})
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    valid_auc = roc_auc_score(labels, out_of_fold)
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission1,submission2, feature_importances, metrics

In [8]:
model(features, test_featurees, encoding = 'ohe', n_folds = 5)

Training Data Shape:  (307511, 338)
Testing Data Shape:  (307511, 338)


NameError: name 'RandomForestClassifier' is not defined